In [ ]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from Utils import MultilayerPerceptron, FGSM

**Reading the Dataset**

In [ ]:
dataset = pd.read_csv('./data.csv')
dataset_label = pd.read_csv('./label.csv')

**Normalizing the data wiith MinMax scaler**

In [ ]:
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)

**Converting the dataset and labels to Torch Tensor and creating the TensorDataset**

In [ ]:
train_dataset = TensorDataset(torch.FloatTensor(dataset), torch.LongTensor(dataset_label.values))

**Loading the trained model**

In [ ]:
criterion = nn.CrossEntropyLoss()
model = MultilayerPerceptron()
model.load_state_dict(torch.load('./DNN.pt'))
model.eval()

**Loading the mask for selecting features of each categories**

In [ ]:
masks = pd.read_csv('./masks.csv')

**Generating adversarial samples for each categories with the desired epsilon value**

In [ ]:
testloader = DataLoader(train_dataset, batch_size=100000, shuffle=False)
categories = ['ALL', 'FWD', 'BWD', 'FL', 'T', 'PH', 'PP', 'FWD_BWD', 'FWD_FL', 'BWD_FL', 'FWD_BWD_FL', 'PH_PP',
              'T_PH_PP']
epsilon = 0.001

for cat in categories:
    print(f'Generating adversarial samples for {cat} features:')
    success = 0
    detected_samples_number = 0
    results = pd.DataFrame(columns=['True', 'Adv'])
    
    for b, (x, y) in enumerate(testloader):
        
        b += 1
        print(f'Batch number: {b:3}')
            
        y_pred = model(x)
        detected_samples_index = (torch.max(y_pred.data, 1)[1] == y).nonzero()
        print(f'Detected samples for Bath number {b:3}: {detected_samples_index.shape[0]:6}')
        detected_samples_number += detected_samples_index.shape[0]
        
        if detected_samples_index.shape[0] == 0:
            print("No sample detected!")
            continue
            
        detected_samples = x[detected_samples_index].reshape(detected_samples_index.shape[0], 76)
        detected_samples_labels = y[detected_samples_index].reshape(detected_samples_index.shape[0])
        y_pred = model(detected_samples)

        adv = FGSM(model, criterion, epsilon, detected_samples.clone().detach().requires_grad_(True), 
                   detected_samples_labels.clone().detach(),  mask=masks[masks['Type'] == cat].iloc[:, 1:].values[0])

        y_adv = model(adv)

        success_adv = (torch.max(y_pred.data, 1)[1] != torch.max(y_adv.data, 1)[1]).nonzero()
        success += success_adv.shape[0]
        
        if success_adv.shape[0] != 0:
            success_pred_data = y_pred[success_adv].reshape(success_adv.shape[0], 15)
            success_adv_data = y_adv[success_adv].reshape(success_adv.shape[0], 15)
            results = results.append(pd.DataFrame({'True': torch.max(success_pred_data.data, 1)[1],
                                                   'Adv': torch.max(success_adv_data.data, 1)[1]}),
                                     ignore_index=True)
        
        print(f'Generated Adv for Bath number {b:3}: {success_adv.shape[0]:8}')
        print(f'All Generated Adv After Bath number {b:3}: {success:8}')
        print()
        
    fname = './' + cat + '_' + str(epsilon)[2:] + '.csv'
    results.to_csv(fname, index=False)